In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from time import sleep
import numpy as np
import cvzone
import pyautogui

In [ ]:
cap=cv2.VideoCapture(0)
detector=HandDetector(detectionCon=0.8)
cap.set(3,1280)
cap.set(4,720)

def drawAll(img,buttonList):
    #tranparency
#     shapes = np.zeros_like(img, np.uint8)
    for button in buttonList:
        x,y=button.pos
        w,h=button.size
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,0),cv2.FILLED)
        cv2.line(img,(x-2,y-2),(x+15,y-2),(255,255,255),2)
        cv2.line(img,(x-2,y-2),(x-2,y+15),(255,255,255),2)
        cv2.line(img,(x+w+2,y-2),(x+w-15,y-2),(255,255,255),2)
        cv2.line(img,(x+w+2,y-2),(x+w+2,y+15),(255,255,255),2)
        cv2.line(img,(x+w+2,y+h+2),(x+w-15,y+h+2),(255,255,255),2)
        cv2.line(img,(x+w+2,y+h+2),(x+w+2,y+h-15),(255,255,255),2)
        cv2.line(img,(x-2,y+h+2),(x-2,y+h-15),(255,255,255),2)
        cv2.line(img,(x-2,y+h+2),(x+15,y+h+2),(255,255,255),2)
        cv2.putText(img,button.text,(x+15,y+65),cv2.FONT_HERSHEY_PLAIN,5,(255,255,255),5)
    return img
        #transparency
#     out = img.copy()
#     alpha = 0.3
#     mask = shapes.astype(bool)
#     out[mask] = cv2.addWeighted(img, alpha, shapes, 1 - alpha, 0)[mask]
#     return out

class button():
    def __init__(self,pos,text,size=[80,80]):
        self.pos=pos
        self.text=text
        self.size=size

keys=[["1","2","3","4","5","6","7","8","9","0"],
      ["Q","W","E","R","T","Y","U","I","O","P"],
      ["A","S","D","F","G","H","J","K","L",";"],
      ["Z","X","C","V","B","N","M",",","/","?"]]

buttonList=[]
for i in range(len(keys)):
    for j,key in enumerate(keys[i]):
        buttonList.append(button([100*j+50,100*i+50],key))

finalText=""

while cap.isOpened():
    r,f=cap.read()
    f=cv2.flip(f,1)
    if r==True:
        hands,img=detector.findHands(f)
        img=drawAll(img,buttonList)   
        if hands:
            hands1=hands[0]
            lmList=hands1["lmList"]
            bbox=hands1["bbox"]
            
            for button in buttonList:
                x,y=button.pos
                w,h=button.size
                
                if (x < lmList[8][0] < (x+w)) & (y < lmList[8][1] < (y+h)):
                    cv2.rectangle(img,(x-7,y-7),(x+w+7,y+h+7),(0,0,0),cv2.FILLED)
                    cv2.putText(img,button.text,(x+15,y+65+7),cv2.FONT_HERSHEY_PLAIN,5,(255,255,255),5)
                    
                    if (((lmList[12][0]-lmList[8][0])**2 + (lmList[12][1]-lmList[8][1])**2)**0.5) < 35:
                        pyautogui.keyDown(button.text)  #pressing the key in the keyboard
                        cv2.rectangle(img,(x-7,y-7),(x+w+7,y+h+7),(0,255,0),cv2.FILLED)
                        cv2.putText(img,button.text,(x+15,y+65+7),cv2.FONT_HERSHEY_PLAIN,5,(255,255,255),5)
                        finalText+=button.text
                        sleep(0.15)
        

        cv2.rectangle(img,(50,450),(700,550),(20,20,20),cv2.FILLED)
        cv2.putText(img,finalText,(60,530),cv2.FONT_HERSHEY_PLAIN,5,(255,255,255),5)
        cv2.imshow("img",img)
        if cv2.waitKey(25) & 0xff==ord("p"):break
    else: break
        

cap.release()
cv2.destroyAllWindows();